## 수원시 공모전 데이터셋
1. 수원 서베이(2023, 2024)
    : 2024 담당
2. SRI 시민패널조사
3. 도시정책지표
4. 데이터분석활용사례

In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel(r".\cmpt_data\1. 수원서베이\12_데이터공모전(공개데이터)\(HRC250604) 2024년 수원서베이 용역_공개용 데이터\(HRC250604) 2024년 수원서베이 용역_공개용 데이터(엑셀).xlsx")
print(df.head())

  응답자 PID (가구용) 문0. 가구 정보 - 1. 거주 주택 유형 (가구용) 문0. 가구 정보 - 2. 1인 가구 여부  \
0     PID                          H0a1                          H0a2   
1   29715                             1                             1   
2   51295                             1                             2   
3   52982                             1                             2   
4   54416                             1                             2   

  (가구용) 문0. 가구 정보 - 3. 맞벌이 여부(가구주 기준) (가구용) 문0. 응답자 성별 (가구용) 문0. 응답자 출생연도  \
0                                H0a3           gender              birth   
1                                   3                1               1992   
2                                   1                2               1981   
3                                   2                1               1977   
4                                   2                1               1975   

  (가구용) 문0. 총 가구원 수 (가구용) 문0. 가구원1 가구대표자(응답자)와의 관계  \
0              H0a4                         

In [2]:
# '교통'이라는 문자열이 포함된 컬럼만 필터링
transport_columns = [col for col in df.columns if '교통' in str(col)]
transport_df = df[transport_columns]

# 결과 출력
print(transport_df.head())


  (가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 무엇입니까?  \
0                                              Q15_2     
1                                                  4     
2                                                  1     
3                                                  3     
4                                                  1     

  (가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 무엇입니까?_기타  \
0                                            Q15_2op        
1                                                NaN        
2                                                NaN        
3                                                NaN        
4                                                NaN        

  (가구대표자용) 문16. 귀하는 평소 이동하실 때 주로 어떠한 교통수단을 이용하십니까?  \
0                                              Q16   
1                                                2   
2                                                1   
3                                                1   
4             

In [7]:
import pandas as pd

# 엑셀 데이터 (header=None 없이 일반 방식으로 불러온 경우)
df = pd.read_excel(r".\cmpt_data\1. 수원서베이\12_데이터공모전(공개데이터)\(HRC250604) 2024년 수원서베이 용역_공개용 데이터\(HRC250604) 2024년 수원서베이 용역_공개용 데이터(엑셀).xlsx")

# step 1: '교통' 포함 컬럼 필터링
transport_cols = [col for col in df.columns if '교통' in str(col)]
transport_df = df[transport_cols]

# step 2: 문항 코드 추출 (각 컬럼의 0번째 행)
question_codes = df.loc[0, transport_cols]

# step 3: 응답 데이터만 추출 (1행부터 시작)
response_data = df.loc[1:, transport_cols].copy()
response_data.reset_index(drop=True, inplace=True)

# step 4: PID 열 추출
pid_col = [col for col in df.columns if 'PID' in col][0]  # '응답자 PID' 등
response_data.insert(0, 'PID', df.loc[1:, pid_col].values)

# step 5: long-format으로 변환
melted = response_data.melt(id_vars='PID', var_name='문항컬럼명', value_name='답변')

# step 6: 문항 코드 매핑
melted['문항코드'] = melted['문항컬럼명'].map(question_codes.to_dict())

# step 7: 필요한 컬럼만 추출
result_df = melted[['PID', '문항코드', '답변']].dropna(subset=['문항코드'])

# 결과 확인
print(result_df.head())


     PID   문항코드 답변
0  29715  Q15_2  4
1  51295  Q15_2  1
2  52982  Q15_2  3
3  54416  Q15_2  1
4  64244  Q15_2  7


In [9]:
print(result_df.info())
print(result_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64197 entries, 0 to 64196
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PID     64197 non-null  object
 1   문항코드    64197 non-null  object
 2   답변      57460 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB
None
(64197, 3)


In [23]:
# 문항코드별 응답값 개수
grouped = result_df.groupby(['문항코드', '답변']).size().reset_index(name='응답자수')
print(grouped)


      문항코드 답변  응답자수
0    Q15_2  1   995
1    Q15_2  2   705
2    Q15_2  3   279
3    Q15_2  4    60
4    Q15_2  5     5
..     ... ..   ...
140  Q60a2  3    78
141  Q60a2  4   403
142  Q60a2  5   792
143  Q60a2  6   969
144  Q60a2  7   804

[145 rows x 3 columns]


In [15]:
df_unique = result_df['문항코드'].unique()
print(df_unique)

['Q15_2' 'Q15_2op' 'Q16' 'Q16op' 'Q17a1' 'Q17a2' 'Q17a3' 'Q17a4' 'Q17a5'
 'Q17a6' 'Q18a1' 'Q18a2' 'Q18a3' 'Q18a4' 'Q18a5' 'Q18a6' 'Q19a1' 'Q19a2'
 'Q36_2a2' 'Q59a10' 'Q60a2']


In [47]:
df_codebook = pd.read_excel(r".\cmpt_data\1. 수원서베이\12_데이터공모전(공개데이터)\(HRC250604) 2024년 수원서베이 용역_공개용 데이터\(HRC250604) 2024년 수원서베이 용역_공개용 데이터_코드북.xlsx")
print(df_codebook.head())

  Variable Name                 Variable Label  Value Code Value Label
0           PID                        응답자 PID         NaN         NaN
1          H0a1  (가구용) 문0. 가구 정보 - 1. 거주 주택 유형         1.0         아파트
2          H0a1  (가구용) 문0. 가구 정보 - 1. 거주 주택 유형         2.0         그 외
3          H0a2  (가구용) 문0. 가구 정보 - 2. 1인 가구 여부         1.0       1인 가구
4          H0a2  (가구용) 문0. 가구 정보 - 2. 1인 가구 여부         2.0       다인 가구


In [48]:
# 결측 제거
df_codebook_clean = df_codebook.dropna(subset=['Variable Name', 'Variable Label'])

# (1) 문항코드 → 문항명 딕셔너리
question_label_dict = df_codebook_clean.drop_duplicates(subset=['Variable Name']) \
    .set_index('Variable Name')['Variable Label'].to_dict()

# (2) (문항코드, 답변코드) → 보기 텍스트 딕셔너리
# Value Code는 문자열로 변환 (grouped에서 비교 위해)
df_codebook_clean['Value Code'] = df_codebook_clean['Value Code'].astype(str)

value_label_dict = {
    (row['Variable Name'], row['Value Code']): row['Value Label']
    for _, row in df_codebook_clean.iterrows()
}

# 결과 확인 (샘플)
print("문항코드 → 문항명:", list(question_label_dict.items())[:3])
print("문항+답변코드 → 보기 텍스트:", list(value_label_dict.items())[:3])


문항코드 → 문항명: [('PID', '응답자 PID'), ('H0a1', '(가구용) 문0. 가구 정보 - 1. 거주 주택 유형'), ('H0a2', '(가구용) 문0. 가구 정보 - 2. 1인 가구 여부')]
문항+답변코드 → 보기 텍스트: [(('PID', 'nan'), nan), (('H0a1', '1.0'), '아파트'), (('H0a1', '2.0'), '그 외')]


In [49]:
from pprint import pprint

pprint(question_label_dict)


{'DM1': '성별',
 'DM10': '주택 유형',
 'DM11': '주택 점유형태',
 'DM12': '가구대표자 직업',
 'DM121': '가구대표자 고용형태',
 'DM122': '가구대표자 사업체 유형',
 'DM13': '월평균 가구소득',
 'DM14': '경제사회적 위치',
 'DM15': '맞벌이 가구 여부',
 'DM16': '가구유형',
 'DM161': '영유아 자녀 가구',
 'DM171': '빈둥지 가구1',
 'DM172': '빈둥지 가구2',
 'DM2': '연령대',
 'DM21': '연령대1',
 'DM22': '연령대2',
 'DM23': '연령대3',
 'DM24': '연령대(19세 구분)',
 'DM25': '세대',
 'DM3': '거주지역',
 'DM31': '거주 구',
 'DM32': '거주 동',
 'DM33': '수원시 생활권',
 'DM4': '가구대표자 학력',
 'DM5': '가구대표자 혼인상태',
 'DM6': '종교',
 'DM7': '수원 거주 기간',
 'DM8': '가구원 수',
 'DM81': '가구원 수1',
 'DM82': '가구원 수2',
 'DM9': '현재 사는 집 거주 기간',
 'H0a1': '(가구용) 문0. 가구 정보 - 1. 거주 주택 유형',
 'H0a10n1': '(가구용) 문0. 가구원1 출생년도',
 'H0a10n2': '(가구용) 문0. 가구원2 출생년도',
 'H0a10n3': '(가구용) 문0. 가구원3 출생년도',
 'H0a10n4': '(가구용) 문0. 가구원4 출생년도',
 'H0a10n5': '(가구용) 문0. 가구원5 출생년도',
 'H0a10n6': '(가구용) 문0. 가구원6 출생년도',
 'H0a10n7': '(가구용) 문0. 가구원7 출생년도',
 'H0a11n1': '(가구용) 문0. 가구원1 혼인상태',
 'H0a11n2': '(가구용) 문0. 가구원2 혼인상태',
 'H0a11n3': '(가구용) 문0. 가구원3 혼인상태',
 'H0a11n

In [50]:
pprint(value_label_dict)

{('DM1', '1.0'): '남자',
 ('DM1', '2.0'): '여자',
 ('DM10', '1.0'): '아파트',
 ('DM10', '2.0'): '단독주택',
 ('DM10', '3.0'): '연립다세대주택',
 ('DM10', '4.0'): '주택 외 거주',
 ('DM11', '1.0'): '자가',
 ('DM11', '2.0'): '전세',
 ('DM11', '3.0'): '월세',
 ('DM11', '4.0'): '기타',
 ('DM12', '1.0'): '관리자',
 ('DM12', '10.0'): '군인',
 ('DM12', '11.0'): '기타',
 ('DM12', '2.0'): '전문가 및 관련 종사자',
 ('DM12', '3.0'): '사무 종사자',
 ('DM12', '4.0'): '서비스 종사자',
 ('DM12', '5.0'): '판매 종사자',
 ('DM12', '6.0'): '농림·어업 숙련 종사자',
 ('DM12', '7.0'): '기능원 및 관련 기능 종사자',
 ('DM12', '8.0'): '장치·기계 조작 및 조립 종사자',
 ('DM12', '9.0'): '단순노무 종사자',
 ('DM12', '99.0'): '무직',
 ('DM121', '1.0'): '상용직 임금근로자',
 ('DM121', '2.0'): '임시직 임금근로자',
 ('DM121', '3.0'): '일용직 임금근로자',
 ('DM121', '4.0'): '특수고용 종사자',
 ('DM121', '5.0'): '고용원이 있는 고용주',
 ('DM121', '6.0'): '고용원이 없는 자영업자',
 ('DM121', '7.0'): '무급가족종사자',
 ('DM121', '99.0'): '무직',
 ('DM122', '1.0'): '대기업',
 ('DM122', '2.0'): '공사 등 공기업',
 ('DM122', '3.0'): '정부, 지방자치단체 기관',
 ('DM122', '4.0'): '중견기업',
 ('DM122', '5.0'):

In [51]:
# 최종 계층형 딕셔너리 생성
from collections import defaultdict

# defaultdict으로 초기화
combined_dict = defaultdict(lambda: {'문항명': None, '보기': {}})

# (1) 문항명 넣기
for q_code, q_label in question_label_dict.items():
    combined_dict[q_code]['문항명'] = q_label

# (2) 보기 넣기
for (q_code, ans_code), ans_label in value_label_dict.items():
    # 누락된 문항코드가 있을 수 있으므로 문항명 없으면 패스하거나 '미정의' 처리
    if q_code not in combined_dict:
        combined_dict[q_code]['문항명'] = '미정의'
    combined_dict[q_code]['보기'][ans_code] = ans_label

# dict로 변환 (pprint용)
combined_dict = dict(combined_dict)


In [52]:
pprint(combined_dict['DM1'])

{'문항명': '성별', '보기': {'1.0': '남자', '2.0': '여자'}}


In [53]:
pprint(combined_dict['Q11a1'])

{'문항명': '(가구대표자용) 문11. 귀하께서는 해당 항목에 대해 어느 정도 수익을 얻고 계십니까? 최근 1개월을 기준으로 세전 소득을 '
        '말씀해주세요. - 1. 근로소득',
 '보기': {'0.0': '소득  없음',
        '1.0': '100만원 미만',
        '2.0': '100~ 200만원 미만',
        '3.0': '200~ 300만원 미만',
        '4.0': '300~ 400만원 미만',
        '5.0': '400~ 500만원 미만',
        '6.0': '500만원 이상'}}


In [54]:
# 1. grouped['답변']을 문자열로 변환 (키 맞추기 위해)
grouped['답변'] = grouped['답변'].astype(str)

# 2. 기존 combined_dict을 확장
for _, row in grouped.iterrows():
    문항코드 = row['문항코드']
    답변코드 = row['답변']
    응답자수 = row['응답자수']

    # 문항코드와 보기 딕셔너리가 존재하는 경우만 처리
    if 문항코드 in combined_dict and 답변코드 in combined_dict[문항코드]['보기']:
        보기텍스트 = combined_dict[문항코드]['보기'][답변코드]

        # 보기텍스트가 문자열이면 dict로 감싸기
        if isinstance(보기텍스트, str):
            combined_dict[문항코드]['보기'][답변코드] = {
                '내용': 보기텍스트,
                '응답자수': 응답자수
            }
        else:
            combined_dict[문항코드]['보기'][답변코드]['응답자수'] = 응답자수

    else:
        # 만약 코드북에는 없는데 grouped에는 있는 경우 → 미정의 처리
        if 문항코드 not in combined_dict:
            combined_dict[문항코드] = {'문항명': '미정의', '보기': {}}
        combined_dict[문항코드]['보기'][답변코드] = {
            '내용': '미정의',
            '응답자수': 응답자수
        }


In [55]:
from pprint import pprint
pprint(combined_dict['Q15_2'])


{'문항명': '(가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 무엇입니까?',
 '보기': {'1': {'내용': '미정의', '응답자수': 995},
        '1.0': '자차(승용차)',
        '2': {'내용': '미정의', '응답자수': 705},
        '2.0': '시내버스, 마을버스',
        '3': {'내용': '미정의', '응답자수': 279},
        '3.0': '지하철',
        '4': {'내용': '미정의', '응답자수': 60},
        '4.0': '통근버스',
        '5': {'내용': '미정의', '응답자수': 5},
        '5.0': '택시',
        '6': {'내용': '미정의', '응답자수': 27},
        '6.0': '자전거(무동력)',
        '7': {'내용': '미정의', '응답자수': 25},
        '7.0': '개인형 이동수단(전동휠, 킥보드, 전기자전거 등)',
        '8': {'내용': '미정의', '응답자수': 312},
        '8.0': '도보 이동',
        '9': {'내용': '미정의', '응답자수': 11},
        '9.0': '기타'}}


In [59]:
# 대상 문항들 전체에 대해 반복
for q_code, q_data in combined_dict.items():
    보기_dict = q_data['보기']
    keys_to_delete = []

    for key in list(보기_dict.keys()):
        if '.' in key:
            # '1.0' 형태일 경우
            int_key = key.split('.')[0]  # '1.0' → '1'

            # int형 키가 있고 dict이면 ('내용' 채워야 함)
            if int_key in 보기_dict and isinstance(보기_dict[int_key], dict):
                보기_dict[int_key]['내용'] = 보기_dict[key]  # '1.0'의 텍스트 넣기
                keys_to_delete.append(key)

    # '1.0', '2.0' 같은 키 삭제
    for key in keys_to_delete:
        del 보기_dict[key]


In [ ]:
from pprint import pprint
pprint(combined_dict['Q15_2'])

{'문항명': '(가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 무엇입니까?',
 '보기': {'1': {'내용': '자차(승용차)', '응답자수': 995},
        '2': {'내용': '시내버스, 마을버스', '응답자수': 705},
        '3': {'내용': '지하철', '응답자수': 279},
        '4': {'내용': '통근버스', '응답자수': 60},
        '5': {'내용': '택시', '응답자수': 5},
        '6': {'내용': '자전거(무동력)', '응답자수': 27},
        '7': {'내용': '개인형 이동수단(전동휠, 킥보드, 전기자전거 등)', '응답자수': 25},
        '8': {'내용': '도보 이동', '응답자수': 312},
        '9': {'내용': '기타', '응답자수': 11}}}


In [64]:
import json

# 1. 저장할 파일 경로 지정
output_path = 'combined_dict.json'

# 2. JSON으로 저장 (ensure_ascii=False → 한글 깨짐 방지, indent=2 → 보기 좋게 들여쓰기)
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(combined_dict, f, ensure_ascii=False, indent=2)

print(f"JSON 파일로 저장 완료: {output_path}")

JSON 파일로 저장 완료: combined_dict.json


In [ ]:
for 문항코드, 내용 in combined_dict.items():
    총합 = sum(
        보기.get('응답자수', 0)
        for 보기 in 내용['보기'].values()
        if isinstance(보기, dict)
    )
    if 총합 > 0:
        print(f"{문항코드} ({내용['문항명']}): 총 {총합}명 응답")

Q15_2 ((가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 무엇입니까?): 총 2419명 응답
Q15_2op ((가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 무엇입니까?_기타): 총 11명 응답
Q16 ((가구대표자용) 문16. 귀하는 평소 이동하실 때 주로 어떠한 교통수단을 이용하십니까?): 총 3057명 응답
Q16op ((가구대표자용) 문16. 귀하는 평소 이동하실 때 주로 어떠한 교통수단을 이용하십니까?_기타): 총 4명 응답
Q17a1 ((가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나 잘 구축되어 있다고 생각하십니까? - 1. 버스): 총 3057명 응답
Q17a2 ((가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나 잘 구축되어 있다고 생각하십니까? - 2. 지하철): 총 3057명 응답
Q17a3 ((가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나 잘 구축되어 있다고 생각하십니까? - 3. 택시): 총 3057명 응답
Q17a4 ((가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나 잘 구축되어 있다고 생각하십니까? - 4. 시외버스/고속버스): 총 3057명 응답
Q17a5 ((가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나 잘 구축되어 있다고 생각하십니까? - 5. 개인형 이동수단): 총 3057명 응답
Q17a6 ((가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나 잘 구축되어 있다고 생각하십니까? - 6. 주차시설): 총 3057명 응답
Q18a1 ((가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하십니까? - 1. 버스): 총 3057명 응답
Q18a2 ((가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하십니까? - 2. 지

In [65]:
# 1. result_df에 등장한 문항코드 고유값
df_unique = result_df['문항코드'].unique()

# 2. combined_dict 기준으로 매칭되는 문항명 추출
matched = [
    {'문항코드': code, '문항명': combined_dict[code]['문항명']}
    for code in df_unique if code in combined_dict
]

# 3. DataFrame으로 변환
df_matched = pd.DataFrame(matched)

# 4. 결과 출력
print(df_matched)

       문항코드                                                문항명
0     Q15_2  (가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 ...
1   Q15_2op  (가구대표자용) 문15-2. 귀하가 통근 또는 통학할 때 주로 이용하는 교통수단은 ...
2       Q16   (가구대표자용) 문16. 귀하는 평소 이동하실 때 주로 어떠한 교통수단을 이용하십니까?
3     Q16op  (가구대표자용) 문16. 귀하는 평소 이동하실 때 주로 어떠한 교통수단을 이용하십니...
4     Q17a1  (가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나...
5     Q17a2  (가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나...
6     Q17a3  (가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나...
7     Q17a4  (가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나...
8     Q17a5  (가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나...
9     Q17a6  (가구대표자용) 문17. 다음의 수원시 교통 수단이나 시설별로 관련 인프라가 얼마나...
10    Q18a1  (가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하십니까?...
11    Q18a2  (가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하십니까?...
12    Q18a3  (가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하십니까?...
13    Q18a4  (가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하십니까?...
14    Q18a5  (가구대표자용) 문18. 다음의 수원시 교통 수단이나 시설별로 얼마나 만족하

### 수원시 공모전 프로젝트

목적: 지하철의 수인분당선처럼 U자형 노선에서는 양 끝 지점의 중간 노선이 없으므로 불편한 점이 있는 것처럼 버스 노선 또한 과잉으로 중복된 노선, 비효율적인 노선 배치 등을 개선시킬 수 없을까에 대한 해답으로 시작

구성: 큰 틀로 2개의 플롯 구성(공개용, 비공개용)
- 시민 참여형 페이지(공개용)
    - 1. 노선 정보 지도 시각화(중복 노선)
    - 2. 일반 시민 불편, 불만, 건의 제안할 수 있도록 게시글 or 메모 작성 가능한 기능
    - 3. 노선별 리뷰 형식으로도 구성
- 내부 공무원 사용 페이지(비공개용)
    - 1. 비공개용 데이터를 다룰 수 있는 기능
    - 2. 새로운 정책 제안, 마련 위한 도구

사용 기술: 
- 데이터 분석, 시각화 (미정)
    - Looker Studio: 구글에서 제공하는 데이터 시각화 도구(구글 드라이브에 csv 넣고 연동 가능)
    - BigQuery: 구글에서 제공하는 클라우드형 DB(페타바이트 단위까지 가능)
    - GA4: 구글 데이터 분석 툴
- 개발

#### 데이터셋 정리
1. 수원 서베이 자료
- 2023 수원 서베이: 도로 교통 설문 답안(코드북 데이터와 묶어 전처리) json파일로 추출(=combined_dict)
    - 
- 2024 수원 서베이: 위와 같음(승환님 작업)

2. SRI 시민패널조사
- q8_3 질문(교통 불편사항)

3. 도시정책지표
- 공개번호60 (대중교통만족도("매우만족" + "약간만족" 응답 비율))